In [1]:
import pandas as pd
import numpy as np
import time
import re

In [2]:
tracking2018 = pd.read_csv("data/tracking2018.csv")
tracking2018

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12777346,2018-09-11T05:25:17.800,123.18,51.73,3.76,1.45,0.29,NaN,NaN,None,NaN,football,NaN,NaN,football,59,2018091001,4033,right
12777347,2018-09-11T05:25:17.900,123.40,51.89,3.19,3.02,0.28,NaN,NaN,None,NaN,football,NaN,NaN,football,60,2018091001,4033,right
12777348,2018-09-11T05:25:18.000,123.55,52.18,2.95,3.88,0.32,NaN,NaN,None,NaN,football,NaN,NaN,football,61,2018091001,4033,right
12777349,2018-09-11T05:25:18.100,123.67,52.50,3.01,4.53,0.34,NaN,NaN,None,NaN,football,NaN,NaN,football,62,2018091001,4033,right


In [3]:
tracking2019 = pd.read_csv("data/tracking2019.csv")
tracking2020 = pd.read_csv("data/tracking2020.csv")

In [ ]:
# create training dataset

# first we define all events where the punt returner has the ball

In [5]:
play_data = pd.read_csv("data/plays.csv")
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull()", engine='python', inplace=False)
returned_plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
14,2018090600,2599,"(10:22) M.Bosher punts 46 yards to PHI 28, Cen...",3,4,16,ATL,Punt,Return,37267.0,...,NaN,NaN,NaN,3,6,NaN,46.0,9.0,37,36
26,2018090900,400,J.Tucker kicks 61 yards from BAL 35 to BUF 4. ...,1,0,0,BAL,Kickoff,Return,39470.0,...,NaN,NaN,NaN,7,0,NaN,61.0,28.0,33,45
27,2018090900,485,(7:25) (Punt formation) C.Bojorquez punts 46 y...,1,4,14,BUF,Punt,Return,46903.0,...,NaN,NaN,NaN,7,0,NaN,46.0,-1.0,47,82
28,2018090900,575,(6:37) (Punt formation) S.Koch punts 63 yards ...,1,4,9,BAL,Punt,Return,31018.0,...,NaN,NaN,NaN,7,0,NaN,63.0,4.0,59,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19961,2021010315,1879,J.Myers kicks 58 yards from SEA 35 to SF 7. A....,3,0,0,SEA,Kickoff,Return,41175.0,...,NaN,NaN,NaN,3,6,NaN,58.0,11.0,47,45
19964,2021010315,2235,"(8:39) M.Dickson punts 43 yards to SF 30, Cent...",3,4,8,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,6,6,NaN,43.0,21.0,22,37
19967,2021010315,2497,"(5:17) M.Dickson punts 52 yards to SF 22, Cent...",3,4,9,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,9,6,NaN,52.0,5.0,47,36
19971,2021010315,3105,J.Myers kicks 63 yards from SEA 35 to SF 2. A....,4,0,0,SEA,Kickoff,Return,41175.0,...,NaN,NaN,NaN,16,12,NaN,63.0,18.0,45,75


In [ ]:
# get all plays where it was returned
#                     passResult = NaN
#                     penaltyYards = NaN
# engine='python' for isnull()
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull()", engine='python', inplace=False)
returned_plays

In [ ]:
# test how determine who has the ball
returned_plays.iloc[1,]

In [ ]:
'''
Cell to run to save training set, where only consider ball-carrier and football as states. No other players.
States also shortened to only x,y positions
'''


pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)
#for each play
for index in range(0, num_plays):
    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
    
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue
    
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    
    all_time_steps = play_df.loc[:,'time'].unique()
    
    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}
        
        time_play_df = play_df.query("time == @time_string", inplace=False)
        
        
        # get expected ball carrier
        ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)
        football_entry = time_play_df.query("displayName == 'football'", inplace=False)
        
        ########
        ## Check to make sure ball returner has the ball. If not, skip
        ########
        ball_carrier_pos = ball_carrier_entry.loc[:,['x','y']].to_numpy()
        football_pos = football_entry.loc[:,['x','y']].to_numpy()
        carrier_football_dist = round(np.linalg.norm(ball_carrier_pos - football_pos),3)
        if carrier_football_dist > FOOTBALL_PLAYER_DIST:
            continue
            
        if (len(football_pos) == 0) or (len(ball_carrier_pos) == 0):
            continue
            
        # add current state
        current_state = np.zeros((1, 4))
        current_state[0, 0:2] = ball_carrier_pos
        current_state[0,2:4] = football_pos
        rowDict['state'] = current_state
        # add future states
        next_state = np.zeros((1, 4))
        next_state[0,0:2] = ball_carrier_entry.loc[:,['next_x','next_y']]
        next_state[0,2:4] = football_entry.loc[:,['next_x','next_y']]
        rowDict['next_state'] = next_state
        # add reward
        rowDict['reward'] = ball_carrier_entry.loc[:,'reward'].iloc[0]
        # can derive action by subtracting next position from current position
        action = current_state[0, 0:2] - next_state[0,0:2]
        rowDict['action'] = action.round(2)
        
        dict_list.append(rowDict)
        
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
simple1_training_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

simple1_training_df.to_csv("simple1_train.csv")


In [ ]:

'''
State vector is 
'''

def return_state(gameId, time, df):
    state_df = df.query('time == @time and gameId == @gameId')
    
    
    
    return state_df

In [ ]:
time = tracking2018.iloc[100,0]
print(time)
gameId = tracking2018.loc[0,'gameId']
print(F"game i: {gameId}")

#return_state(gameId, time, tracking2018)
max_frame = max(tracking2018.query("playId == 36 and gameId == @gameId")['frameId'])
print(max_frame)
tracking2018.query("frameId == 11 and playId == 36 and gameId == @gameId")

In [ ]:
'''
Cell to run to save training set, where consider all states and players. Ball carrier listed first, football last
'''

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)
#for each play
for index in range(0, num_plays):
    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
    
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue
    
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    
    all_time_steps = play_df.loc[:,'time'].unique()
    
    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}
        
        time_play_df = play_df.query("time == @time_string", inplace=False)
        
        # sort df to put ball carrier first, football last
        time_play_df['sort'] = pd.Categorical(time_play_df['nflId'], [returnerId])
        time_play_df.sort_values("sort", na_position='last', inplace=True)
        

        states = time_play_df.loc[:,['x','y', 's', 'a', 'o', 'dir']].to_numpy().reshape(-1)
        ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)

        ########
        ## Check to make sure ball returner has the ball. If not, skip
        ########
        ball_carrier_pos = states[0:2]
        football_pos = states[-6:-4]
        carrier_football_dist = round(np.linalg.norm(ball_carrier_pos - football_pos),3)
        if carrier_football_dist > FOOTBALL_PLAYER_DIST:
            continue
        if (len(football_pos) == 0) or (len(ball_carrier_pos) == 0):
            continue
            
        # add current state
        rowDict['state'] = states
        # add future states
        next_state = time_play_df.loc[:,['next_x','next_y', 'next_s', 'next_a', 'next_o', 'next_dir']].to_numpy().reshape(-1)
        rowDict['next_state'] = next_state
        # add reward
        rowDict['reward'] = ball_carrier_entry.loc[:,'reward'].iloc[0]
        # can derive action by subtracting next position from current position
        action = states[0:6] - next_state[0:6]
        rowDict['action'] = action.round(2)
        
        dict_list.append(rowDict)
        
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
full_train_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

full_train_df.to_csv("all_actions_all_players.csv")


In [ ]:
'''
Cell to run to save training set, where consider (x,y) states for all players. Ball carrier listed first, football last
'''

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

#for each play
for index in range(0, num_plays):
    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
    
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue
    
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    
    all_time_steps = play_df.loc[:,'time'].unique()
    
    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}
           
        time_play_df = play_df.query("time == @time_string", inplace=False)
        time_play_df.query("not reward.isnull()", engine='python', inplace=True)
        
        # sort df to put ball carrier first, football last
        time_play_df['sort'] = pd.Categorical(time_play_df['nflId'], [returnerId])
        time_play_df.sort_values("sort", na_position='last', inplace=True)
        

        states = time_play_df.loc[:,['x','y']].to_numpy().reshape(-1)
        ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)

        ########
        ## Check to make sure ball returner has the ball. If not, skip
        ########
        ball_carrier_pos = states[0:2]
        football_pos = states[-2:]
        carrier_football_dist = round(np.linalg.norm(ball_carrier_pos - football_pos),3)
        if carrier_football_dist > FOOTBALL_PLAYER_DIST:
            continue
        if (len(football_pos) == 0) or (len(ball_carrier_pos) == 0):
            continue
            
        rowDict["playIndex"] = index
        rowDict["timeIndex"] = time_index
        # add current state
        rowDict['state'] = states
        # add future states
        next_state = time_play_df.loc[:,['next_x','next_y']].to_numpy().reshape(-1)
        rowDict['next_state'] = next_state
        # add reward
        try:
            rowDict['reward'] = ball_carrier_entry.loc[:,'reward'].iloc[0]
        except:
            print("error getting reward")
            print(ball_carrier_entry)
            continue
        # can derive action by subtracting next position from current position
        action = states[0:2] - next_state[0:2]
        rowDict['action'] = action.round(2)
        
        dict_list.append(rowDict)
        
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
full_train_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

full_train_df.to_csv("all_players.csv")

In [13]:
saved = pd.read_csv('all_players.csv', index_col=0)
saved.head()

,playIndex,timeIndex,state,next_state,reward,action
0,0,48,[11.4 42.67 55.74 36.49 47.81 23.35 47.33 20....,[11.39 42.66 55.07 36.68 46.88 23.75 46.48 20....,0.01,[0.01 0.01]
1,0,49,[11.39 42.66 55.07 36.68 46.88 23.75 46.48 20....,[11.41 42.63 54.4 36.86 45.96 24.14 45.62 21....,0.04,[-0.02 0.03]
2,0,50,[11.41 42.63 54.4 36.86 45.96 24.14 45.62 21....,[11.45 42.56 53.73 37.04 45.04 24.53 44.76 21....,0.08,[-0.04 0.07]
3,0,51,[11.45 42.56 53.73 37.04 45.04 24.53 44.76 21....,[11.53 42.45 53.07 37.21 44.11 24.92 43.9 21....,0.14,[-0.08 0.11]
4,0,52,[11.53 42.45 53.07 37.21 44.11 24.92 43.9 21....,[11.62 42.32 52.41 37.37 43.17 25.29 43.03 22....,0.16,[-0.09 0.13]


In [14]:
saved.query("reward.isnull()", engine='python', inplace=False)

,playIndex,timeIndex,state,next_state,reward,action
94,1,120,[72.69 31.17 58.59 29.91 66.79 42.24 79.13 33....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
145,2,104,[76.7 37.03 80.39 30.21 76.43 42.86 76.43 43....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
211,3,139,[34.57 43.74 33.76 44.96 36.12 44.59 35.56 45....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
319,5,138,[43.13 43.26 42.74 42.18 44.91 43.61 43.63 40....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
438,6,185,[29.82 6.44 32.87 7.88 50.08 15.7 51.38 10....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
...,...,...,...,...,...,...
234451,4389,97,[101. 18.97 83.43 19.95 101.8 22.64 79...,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
234506,4390,94,[88.17 51.95 81.64 45.6 87.09 51.52 85.36 39....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
234544,4391,117,[43.24 44.91 46.35 43.98 46.53 40.97 51.31 27....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]
234582,4392,113,[41.84 56.64 17.3 51.15 48.41 40.01 51.57 34....,[nan nan nan nan nan nan nan nan nan nan nan n...,NaN,[nan nan]


In [67]:
def get_distances(x, player_pos):
    dist = round(np.linalg.norm(player_pos.values - x.values),3)
    return dist

In [116]:

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")

    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")


    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")


    #####################

    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)

    # get starting time of play
    starting_time = min(play_df['time'])

    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        time_play_df = play_df.query("time == @time_string", inplace=False)
        
        # check to make sure there is next state
        null_rewards = time_play_df.query("reward.isna()", engine='python', inplace=False)
        if (len(null_rewards) > 0):
            continue
        

        # sort df to put ball carrier first, football last
        #time_play_df['sort'] = pd.Categorical(time_play_df['nflId'], [returnerId])

        #ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)
        ball_carrier_pos = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y']]

        time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_pos,))

        #time_play_df.sort_values("sort", na_position='last', inplace=True)

        time_play_df.sort_values("dist_from_ball_carrier", inplace=True, ignore_index=True)

        #### 
        #Check to make sure ball carrier has football
        #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
        ####
        football_index = time_play_df.index[time_play_df.team == 'football']
        try:
            football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
        except:
            football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']

        if (football_index != 1) | (football_distance > FOOTBALL_PLAYER_DIST):
            continue

        states = time_play_df.loc[:,['x','y']].to_numpy().reshape(-1)

        rowDict["playIndex"] = index
        rowDict["timeIndex"] = time_index
        # add current state
        rowDict['state'] = states
        # add future states
        next_state = time_play_df.loc[:,['next_x','next_y']].to_numpy().reshape(-1)
        rowDict['next_state'] = next_state
        # add reward
        try:
            rowDict['reward'] = time_play_df.loc[0,'reward']
        except:
            print("error getting reward")
            continue
        # can derive action by subtracting next position from current position
        action = states[0:2] - next_state[0:2]
        rowDict['action'] = action.round(2)

        dict_list.append(rowDict)
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
full_train_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("all_players.pkl")

print(F"total time: {total_time}  seconds")

Play 0 out of 4494
Returner id is null is for index 35 at game 2018090903 for play 4162
Returner id is null is for index 161 at game 2018091700 for play 4233
Returner id is null is for index 198 at game 2018092303 for play 3993
Play 250 out of 4494
Returner id is null is for index 353 at game 2018100400 for play 4630
Could not get returner id for index 362 at game 2018100701 for play 4281: <class 'Exception'>
Returner id is null is for index 387 at game 2018100706 for play 3861
Returner id is null is for index 401 at game 2018100708 for play 3388
Returner id is null is for index 408 at game 2018100709 for play 3866
Returner id is null is for index 413 at game 2018100710 for play 4640
Returner id is null is for index 486 at game 2018101406 for play 4443
Play 500 out of 4494
Could not get returner id for index 502 at game 2018101410 for play 2172: <class 'Exception'>
Returner id is null is for index 542 at game 2018102100 for play 3461
Returner id is null is for index 549 at game 2018102

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().